# PROBLE 1 : SPARK STREAMING
Create an Apache Spark notebook for handling file stream inside a folder.
- Use the data given in news folder **(news-1.json, news-2.json, news-3.json)**
- Put the input inside folder named **input-your-student-id**. Example: **input-51231132**
- Put the output inside folder named **output-your-student-id**

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gdown
import os

url = "https://drive.google.com/drive/folders/19P90u0lkUgaUSWopSu-rJBcsXZvPcdGy?usp=share_link"
os.makedirs("input-5025201253")
gdown.download_folder(url, output="input-5025201253", quiet=True, use_cookies=False)

['input-5025201253/news-1.json',
 'input-5025201253/news-2.json',
 'input-5025201253/news-3.json']

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=607a408b147e01772fe920c23315c852fc9e55d40b0cef1009d632bd149937f8
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [21]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming Process Files") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .master("local[*]") \
    .getOrCreate()

spark

In [22]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)


# Create the streaming_df to read from input directory
streaming_df = spark.readStream\
    .format("json") \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "/content/output-5025201253/") \
    .option("maxFilesPerTrigger", 1) \
    .load("/content/input-5025201253/")

In [25]:
# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
# streaming_df.show(truncate=False)

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)



AnalysisException: ignored

In [29]:
from pyspark.sql.functions import explode, col

exploded_df = streaming_df \
    .select("link", "headline", "short_description", "authors",)

In [30]:
exploded_df.printSchema()

root
 |-- link: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- authors: string (nullable = true)



In [31]:
# Write the output to console sink to check the output
writing_df = exploded_df.writeStream \
    .format("json") \
    .option("path", "content/output-5025201253/") \
    .option("checkpointLocation","checkpoint_dir") \
    .outputMode("append") \
    .start()
    
# Start the streaming application to run until the following happens
# 1. Exception in the running program
# 2. Manual Interruption
writing_df.awaitTermination()



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored